<a href="https://colab.research.google.com/github/shredder0812/VXLNhom12/blob/test/edone_ocsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp /content/drive/MyDrive/Daday_3.mp4 /content/drive/MyDrive/best.pt /content

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics
!git clone https://github.com/KeeganFernandesWork/yolo_tracking
%cd yolo_tracking
!pip install -r requirements.txt
!pip install .

In [2]:
from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT,
                    create_tracker, get_tracker_config)
from pathlib import Path
import cv2
import sys
import numpy as np
import datetime
from ultralytics import YOLO

Video Writer

In [3]:
# sys.path.insert(1, '/kaggle/input/video-weights')

def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

In [7]:
color = (0, 0, 255)  # BGR
thickness = 2
fontscale = 0.5

device = "cuda:0" # cuda:0 , cpu
fp16 = True # True if gpu available
# load the pre-trained YOLOv8n model
model = YOLO("/content/best.pt")
source = "/content/Daday_3.mp4"

Testing DeepOCSORT

In [10]:
help(DeepOCSORT)

Help on class DeepOCSort in module boxmot.trackers.deepocsort.deep_ocsort:

class DeepOCSort(builtins.object)
 |  DeepOCSort(model_weights, device, fp16, per_class=True, det_thresh=0.3, max_age=30, min_hits=3, iou_threshold=0.3, delta_t=3, asso_func='iou', inertia=0.2, w_association_emb=0.75, alpha_fixed_emb=0.95, aw_param=0.5, embedding_off=False, cmc_off=False, aw_off=False, new_kf_off=False, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model_weights, device, fp16, per_class=True, det_thresh=0.3, max_age=30, min_hits=3, iou_threshold=0.3, delta_t=3, asso_func='iou', inertia=0.2, w_association_emb=0.75, alpha_fixed_emb=0.95, aw_param=0.5, embedding_off=False, cmc_off=False, aw_off=False, new_kf_off=False, **kwargs)
 |      Sets key parameters for SORT
 |  
 |  dump_cache(self)
 |  
 |  update = wrapper(*args, **kwargs)
 |  
 |  update_public(self, dets, cates, scores)
 |  
 |  ----------------------------------------------------------------------
 |  Data descripto

In [21]:
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device=device,
    fp16=fp16,
)
vid = cv2.VideoCapture(source)
writer = create_video_writer(vid, "DeepOCSORT.mp4")

while True:
    ret, im = vid.read()

    detections = model.predict(im )[0]






    # initialize the list of bounding boxes and confidences
    results = []
    if not ret:
        break
    print(np.array(detections.boxes.data.tolist()).ndim) # 2 -> polyp, 1 -> no detect
    #print(results)



    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273],[ 0]]

    ts = tracker.update(np.array(detections.boxes.data.tolist()), im) # --> (x, y, x, y, id, conf, cls)

    xyxys = ts[:,0:4].astype('int') # float64 to int
    ids = ts[:, 4].astype('int') # float64 to int
    confs = ts[:, 5]
    clss = ts[:, 6]

    # print bboxes with their associated id, cls and conf
    if ts.shape[0] != 0:
        for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
            im = cv2.rectangle(
                im,
                (xyxy[0], xyxy[1]),
                (xyxy[2], xyxy[3]),
                color,
                thickness
            )
            cv2.putText(
                im,
                f'id: {id}, conf: {conf}, c: {cls}',
                (xyxy[0], xyxy[1]-10),
                cv2.FONT_HERSHEY_SIMPLEX,
                fontscale,
                color,
                thickness
            )

    # show the frame to our screen

    writer.write(im)


vid.release()
writer.release()
print("Task Completed")

2024-02-22 15:22:04.061 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:187 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-02-22 15:22:04.063 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:191 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')



0: 512x640 1 polyp, 73.7ms
Speed: 3.2ms preprocess, 73.7ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 1 polyp, 72.6ms
Speed: 2.5ms preprocess, 72.6ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 1 polyp, 72.4ms
Speed: 2.3ms preprocess, 72.4ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 1 polyp, 71.0ms
Speed: 2.5ms preprocess, 71.0ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 1 polyp, 69.4ms
Speed: 2.2ms preprocess, 69.4ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 1 polyp, 64.0ms
Speed: 2.2ms preprocess, 64.0ms inference, 1.7ms postprocess per image at shape (1, 3, 512, 640)
2

0: 512x640 (no detections), 59.8ms
Speed: 2.6ms preprocess, 59.8ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)
1


AssertionError: Unsupported 'dets' dimensions, valid number of dimensions is two